<a href="https://colab.research.google.com/github/orlandoalexander/Moda-Personal-Assistant/blob/models/Attribute_ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Input
import pandas as pd
import numpy as np


In [4]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install git+https://github.com/orlandoalexander/Moda-Personal-Assistant.git@preproc_package


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/orlandoalexander/Moda-Personal-Assistant.git (to revision preproc_package) to /tmp/pip-req-build-jrb2upwy
  Running command git clone -q https://github.com/orlandoalexander/Moda-Personal-Assistant.git /tmp/pip-req-build-jrb2upwy
  Running command git checkout -b preproc_package --track origin/preproc_package
  Switched to a new branch 'preproc_package'
  Branch 'preproc_package' set up to track remote branch 'preproc_package' from 'origin'.
     |████████████████████████████████| 11.7 MB 3.7 MB/s 
     |████████████████████████████████| 17.1 MB 72.7 MB/s 
     |████████████████████████████████| 11.3 MB 83.6 MB/s 
     |████████████████████████████████| 4.5 MB 56.2 MB/s 
     |████████████████████████████████| 61 kB 64 kB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 31.2 MB 1.3 MB/s 
     |██████████████████

In [ ]:
# !unzip '/content/drive/MyDrive/attribute_img/img.zip''
!unzip '/content/drive/MyDrive/Le Wagon/Moda-Personal-Assistant/Data/img.zip'

In [3]:
from preproc.preprocess import AttributePreproc


In [4]:
prep = AttributePreproc('img', (224,224), 'sleeves', 0.2) 

In [5]:
X_train, X_test, y_train, y_test  = prep.run()

Augmenting attribute 'long_sleeve'...
Augmenting attribute 'short_sleeve'...
Augmenting attribute 'sleeveless'...
Done!


In [6]:
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Lambda
from keras import Sequential
from keras.metrics import Precision, Recall
from keras.optimizers import Adam

from keras.applications import InceptionV3
from keras.applications import inception_v3
from keras.applications import ResNet50
from keras.applications import MobileNetV2
from keras.applications import MobileNet
from keras.applications import mobilenet_v2
from keras.applications import EfficientNetB0
from keras.applications import efficientnet
from keras.applications import mobilenet
from keras.applications import resnet
from keras.applications import inception_v3

# attribute should be one of the following strings:
# 'design', 'sleeves', 'length', 'neckline', 'fabric', 'fit'

# model should be one of the following strings:
# 'inception', 'resnet', 'mobilenet', 'efficientnet'

class AttrModel:
    def __init__(self, attribute, epochs, model, input_shape, batch_size, final_layer_neurons,
                 X_train, X_test,
                 y_train, y_test, **kwargs):
        self.attribute = attribute
        self.model = model.lower()
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.final_layer_neurons = final_layer_neurons
        self.kwargs = kwargs
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            X_train, y_train, test_size=0.5
        )

        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.cat_nums = {
            'design': 7,
            'sleeves': 3,
            'length': 2,
            'neckline': 4,
            'fabric': 6,
            'fit': 3
        }

        del X_train
        del y_train

        self.cat_num = self.cat_nums[self.attribute]
        self.activation = 'sigmoid' if self.attribute == 'length' else 'softmax'
        self.loss = 'binary_crossentropy' if self.attribute == 'length' else 'categorical_crossentropy'
        self.model = self.instantiate_model()  # calling the function below

    def instantiate_model(self):
        input = Input(self.input_shape)
        #preprocess = Lambda(mobilenet.preprocess_input)

        if self.model == 'inception':              # calling the chosen pretrained model
            base_model = InceptionV3(include_top=False, weights='imagenet',
                                    classes=self.cat_num, input_tensor=input)
            self.preprocess_input = inception_v3.preprocess_input
        elif self.model == 'resnet':
            base_model = ResNet50(include_top=False, weights='imagenet',
                                  classes=self.cat_num, input_tensore=input)
            self.preprocess_input = resnet.preprocess_input
        elif self.model == 'mobilenet':
            base_model = MobileNetV2(include_top=False, weights='imagenet',
                                     classes=self.cat_num, input_tensor=input)
            self.preprocess_input = mobilenet.preprocess_input
        elif self.model == 'efficientnet':
            base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                        classes=self.cat_num, input_tensor=input)
            self.preprocess_input = efficientnet.preprocess_input
        else:
            print('''No model found. Please pass one of the following:
                  inception, resnet, mobilenet, efficientnet''')

        base_model.trainable = False    # freeze layers
        pool = GlobalAveragePooling2D()
        dense = Dense(units=self.final_layer_neurons, activation='relu')
        prediction = Dense(units=self.cat_num, activation=self.activation)
        model = Sequential([base_model, pool, dense, prediction])
        print(self.loss) # check loss func
        model.compile(loss=self.loss, optimizer='adam',
                      metrics=['accuracy', Precision(), Recall()])
        return model

    def train(self):
        datagen = ImageDataGenerator(preprocessing_function=self.preprocess_input)

        self.train_generator = datagen.flow( 
        self.X_train, self.y_train,                                
        batch_size=16)

        self.val_generator = datagen.flow( 
        self.X_val, self.y_val,                                
        batch_size=16)

        self.test_generator = datagen.flow( 
        self.X_test, self.y_test,                                
        batch_size=16)

        # Train the model on the images and labels
        self.model.history = self.model.fit(self.train_generator, 
        validation_data = self.val_generator,
        epochs=self.epochs, batch_size=self.batch_size, verbose=1, 
        callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])

        return self.model.history

    def finetune(self,callbacks=[EarlyStopping(patience=3)],
                 loss='categorical_crossentropy',
                 metrics=['accuracy', Precision(), Recall()]):
      
        self.model.trainable = True     # unfreeze layers, then compile to save changes
        self.model.compile(
            optimizer=Adam(1e-5),                  # Very low learning rate
            loss=loss,
            metrics=metrics)
        self.model.fit(self.val_generator,
                       epochs=self.epochs, batch_size=self.batch_size,
                       callbacks=[EarlyStopping(patience=2, monitor='val_accuracy', restore_best_weights=True)],
                       validation_split=0.2)
        return self.model.history

    def evaluate(self):
        return self.model.evaluate(self.test_generator, verbose=1)

    def predict(self, X):
        return self.model.predict(X)


In [7]:
mobilnet = AttrModel(attribute='sleeves',
                      model='mobilenet',
                      input_shape=(224,224,3),
                      batch_size= 16,
                      epochs=50,
                      final_layer_neurons=100,
                      X_train=X_train,
                      X_test=X_test,
                      y_train=y_train,
                      y_test=y_test)

categorical_crossentropy


In [8]:
mobilnet.train()

Epoch 1/50
810/810 [==============================] - 41s 37ms/step - loss: 0.4886 - accuracy: 0.8057 - precision_1: 0.8302 - recall_1: 0.7774 - val_loss: 0.3787 - val_accuracy: 0.8491 - val_precision_1: 0.8677 - val_recall_1: 0.8340
Epoch 2/50
810/810 [==============================] - 29s 36ms/step - loss: 0.3832 - accuracy: 0.8519 - precision_1: 0.8690 - recall_1: 0.8339 - val_loss: 0.3495 - val_accuracy: 0.8591 - val_precision_1: 0.8769 - val_recall_1: 0.8404
Epoch 3/50
810/810 [==============================] - 29s 35ms/step - loss: 0.3360 - accuracy: 0.8720 - precision_1: 0.8870 - recall_1: 0.8570 - val_loss: 0.2885 - val_accuracy: 0.8920 - val_precision_1: 0.9052 - val_recall_1: 0.8746
Epoch 4/50
810/810 [==============================] - 29s 35ms/step - loss: 0.2914 - accuracy: 0.8906 - precision_1: 0.9036 - recall_1: 0.8769 - val_loss: 0.2478 - val_accuracy: 0.9065 - val_precision_1: 0.9174 - val_recall_1: 0.8988
Epoch 5/50
810/810 [==============================] - 29s 35ms/s

KeyboardInterrupt: ignored

In [ ]:
mobilnet.save() # SAVE MODEL